In [1]:
!pip install -r requirements.txt
!pip install datasets trl peft bitsandbytes wandb accelerate 'transformers[torch]' torch


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install scikit-multilearn


[notice] A new release of pip is available: 23.0.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd

In [2]:
import os
import functools
import numpy as np
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from datasets import Dataset, DatasetDict
import torch
import torch.nn.functional as F
from peft import (
	LoraConfig,
	prepare_model_for_kbit_training,
	get_peft_model
)
from transformers import (
	AutoModelForSequenceClassification,
	AutoTokenizer,
	BitsAndBytesConfig,
	TrainingArguments,
	Trainer
)

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
path1 = "train_opp_annotations.csv"
path2 = "policy_train_data.csv"
df1 = pd.read_csv(path1, sep="	")
df2 = pd.read_csv(path2, sep="	")

In [6]:
df1

,Folder,DocID,QueryID,Split,Query,first,third,datasecurity,dataretention,user_access,user_choice,other
0,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,train,do you keep the data of mine and upload to you...,0,0,0,1,0,0,0
1,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_1,train,do i need to pay,0,0,0,0,0,0,0
2,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_2,train,do you know and use my location,1,0,0,0,0,0,0
3,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_3,train,do you sell my data,0,1,0,0,0,0,0
4,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_4,train,is there any incognito mode,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1345,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_45,train,do i have to sign in using a social media acco...,0,0,0,0,0,0,0
1346,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_46,train,will any academic institutions or employers be...,0,1,0,0,0,0,0
1347,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_47,train,are potential employers given access to a user...,0,1,0,0,0,0,0
1348,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_48,train,which third party entities will have access to...,0,1,0,0,0,0,0


In [7]:
df2

,Folder,DocID,QueryID,SentID,Split,Query,Segment,Label
0,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,Cake Web Browser _1_0_0,train,do you keep the data of mine and upload to you...,"This privacy policy, with our Terms of Servic...",Irrelevant
1,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,Cake Web Browser _1_0_1,train,do you keep the data of mine and upload to you...,We encourage you to read this privacy policy c...,Irrelevant
2,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,Cake Web Browser _1_0_2,train,do you keep the data of mine and upload to you...,By using our application or other online servi...,Irrelevant
3,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,Cake Web Browser _1_0_3,train,do you keep the data of mine and upload to you...,"When we post changes to this privacy policy, ...",Irrelevant
4,../../Dataset/Train/com.cake.browser,Cake Web Browser _1,Cake Web Browser _1_0,Cake Web Browser _1_0_4,train,do you keep the data of mine and upload to you...,We encourage you to review this privacy policy...,Irrelevant
...,...,...,...,...,...,...,...,...
185195,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_49,Khan Academy _27_49_341,train,does the app track my location?,"As a Parent, you have the ability to access co...",Irrelevant
185196,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_49,Khan Academy _27_49_342,train,does the app track my location?,To refuse future collection or use of your chi...,Irrelevant
185197,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_49,Khan Academy _27_49_343,train,does the app track my location?,"Please note, you must delete your Child Users ...",Irrelevant
185198,../../Dataset/Train/org.khanacademy.android,Khan Academy _27,Khan Academy _27_49,Khan Academy _27_49_344,train,does the app track my location?,If you do not have a Parent Account associated...,Irrelevant


In [4]:
merged_df = pd.merge(df1, df2, on="QueryID")
merged_df = merged_df[["DocID_x", "Query_x", "Segment", "first", "third", "datasecurity", "dataretention", "user_access", "user_choice", "other", "Label"]]
merged_df = merged_df.rename(columns={"DocID_x": "DocID", "Query_x": "Query"})
merged_df["DocID"] = merged_df["DocID"].str.split("_").str[0]
merged_df

,DocID,Query,Segment,first,third,datasecurity,dataretention,user_access,user_choice,other,Label
0,Cake Web Browser,do you keep the data of mine and upload to you...,"This privacy policy, with our Terms of Servic...",0,0,0,1,0,0,0,Irrelevant
1,Cake Web Browser,do you keep the data of mine and upload to you...,We encourage you to read this privacy policy c...,0,0,0,1,0,0,0,Irrelevant
2,Cake Web Browser,do you keep the data of mine and upload to you...,By using our application or other online servi...,0,0,0,1,0,0,0,Irrelevant
3,Cake Web Browser,do you keep the data of mine and upload to you...,"When we post changes to this privacy policy, ...",0,0,0,1,0,0,0,Irrelevant
4,Cake Web Browser,do you keep the data of mine and upload to you...,We encourage you to review this privacy policy...,0,0,0,1,0,0,0,Irrelevant
...,...,...,...,...,...,...,...,...,...,...,...
185195,Khan Academy,does the app track my location?,"As a Parent, you have the ability to access co...",1,0,0,0,0,0,0,Irrelevant
185196,Khan Academy,does the app track my location?,To refuse future collection or use of your chi...,1,0,0,0,0,0,0,Irrelevant
185197,Khan Academy,does the app track my location?,"Please note, you must delete your Child Users ...",1,0,0,0,0,0,0,Irrelevant
185198,Khan Academy,does the app track my location?,If you do not have a Parent Account associated...,1,0,0,0,0,0,0,Irrelevant


In [5]:
merged_df = merged_df[merged_df["Label"] != "Irrelevant"]
merged_df = merged_df.drop(["Label"], axis=1)
# merged_df["Label"] = merged_df[['first', 'third', 'datasecurity', 'dataretention', 'user_access', 'user_choice', 'other']].idxmax(axis=1)
# category_dict = {
# 	'first':'First Party Collection',
# 	'third':'Third Party Sharing',
# 	'datasecurity':'Data Security',
# 	'dataretention':'Data Retention',
# 	'user_choice':'User Choice/Control',
# 	'user_access':'User Access, Edit and Deletion',
# 	'other':'Other'
# }
# merged_df["Label"] = merged_df["Label"].map(category_dict)
# merged_df = merged_df.drop(list(category_dict.keys()), axis=1)

In [6]:
merged_df

,DocID,Query,Segment,first,third,datasecurity,dataretention,user_access,user_choice,other
5,Cake Web Browser,do you keep the data of mine and upload to you...,"We collect, store and use your data in order ...",0,0,0,1,0,0,0
165,Cake Web Browser,do you know and use my location,"We collect, store and use your data in order ...",1,0,0,0,0,0,0
176,Cake Web Browser,do you know and use my location,"In normal browsing mode, the Cake browser may...",1,0,0,0,0,0,0
186,Cake Web Browser,do you know and use my location,We may also collect your geo-location informat...,1,0,0,0,0,0,0
193,Cake Web Browser,do you know and use my location,"We may ask for app permissions (for example, w...",1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
184629,Khan Academy,which third party entities will have access to...,We may share data in the context of a change o...,0,1,0,0,0,0,0
184630,Khan Academy,which third party entities will have access to...,In the event that Khan Academy is involved in ...,0,1,0,0,0,0,0
184631,Khan Academy,which third party entities will have access to...,If the transaction involves the transfer of St...,0,1,0,0,0,0,0
184633,Khan Academy,which third party entities will have access to...,Khan Academy may release Personal Information ...,0,1,0,0,0,0,0


In [7]:
import numpy as np
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split

In [8]:
Q = merged_df[["DocID", "Query", "Segment"]].values
Y = merged_df[["first", "third", "datasecurity", "dataretention", "user_access", "user_choice", "other"]].values

X = ['Query: ' + qs[1] + ', Segment: ' + qs[2] for qs in Q]
X = np.array(X)
labels = np.array(Y, dtype=int)
label_weights = 1 - labels.sum(axis=0) / labels.sum()

row_ids = np.arange(len(Y))

train_idx, Y_train, val_idx, Y_test = iterative_train_test_split(row_ids[:, np.newaxis], labels, test_size=0.2)

X_train = [X[i] for i in train_idx.flatten()]
X_test = [X[i] for i in val_idx.flatten()]

In [9]:
from datasets import Dataset, DatasetDict

ds = DatasetDict({
	"train": Dataset.from_dict({"text": X_train, "labels": Y_train}),
 	"test": Dataset.from_dict({"text": X_test, "labels": Y_test})
})


In [10]:
# define custom batch preprocessor
def collate_fn(batch, tokenizer):
	dict_keys = ['input_ids', 'attention_mask', 'labels']
	d = {k: [dic[k] for dic in batch] for k in dict_keys}
	d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
		d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
	)
	d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
		d['attention_mask'], batch_first=True, padding_value=0
	)
	d['labels'] = torch.stack(d['labels'])
	return d

# define which metrics to compute for evaluation
def compute_metrics(p):
	predictions, labels = p
	f1_micro = f1_score(labels, predictions > 0, average = 'micro')
	f1_macro = f1_score(labels, predictions > 0, average = 'macro')
	f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
	return {
		'f1_micro': f1_micro,
		'f1_macro': f1_macro,
		'f1_weighted': f1_weighted
	}

In [16]:
# model name
model_name = 'meta-llama/Meta-Llama-3-8B'

# preprocess dataset with tokenizer
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

# qunatization config
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True, # enable 4-bit quantization
    bnb_4bit_quant_type = 'nf4', # information theoretically optimal dtype for normally distributed weights
    bnb_4bit_use_double_quant = True, # quantize quantized weights //insert xzibit meme
    bnb_4bit_compute_dtype = torch.bfloat16 # optimized fp format for ML
)

# lora config
lora_config = LoraConfig(
    r = 16, # the dimension of the low-rank matrices
    lora_alpha = 8, # scaling factor for LoRA activations vs pre-trained weight activations
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'],
    lora_dropout = 0.05, # dropout probability of the LoRA layers
    bias = 'none', # wether to train bias weights, set to 'none' for attention layers
    task_type = 'SEQ_CLS'
)

# load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    num_labels=labels.shape[1]
)
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/5709 [00:00<?, ? examples/s]

Map:   0%|          | 0/1428 [00:00<?, ? examples/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from custom_trainer import CustomTrainer

# define training args
training_args = TrainingArguments(
    output_dir='multilabel_classification',
    learning_rate=5e-4,  # Consider testing 2e-4 or 5e-4 if increasing batch size significantly
    per_device_train_batch_size=50,  # Adjust based on trial and observed memory usage
    per_device_eval_batch_size=30,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True
)

# train
trainer = CustomTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds['train'],
    eval_dataset = tokenized_ds['test'],
    tokenizer = tokenizer,
    data_collator = functools.partial(collate_fn, tokenizer=tokenizer),
    compute_metrics = compute_metrics,
    label_weights = torch.tensor(label_weights, device=model.device)
)

trainer.train()

/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,F1 Weighted
1,No log,0.050096,0.930914,0.866617,0.930022
2,No log,0.039655,0.949055,0.905897,0.948345
3,No log,0.017377,0.979631,0.986005,0.979624
4,No log,0.015941,0.981772,0.986265,0.981699
5,0.045000,0.015154,0.981725,0.982362,0.981669
6,0.045000,0.013451,0.983711,0.982350,0.983678
7,0.045000,0.012750,0.984625,0.984079,0.984578
8,0.045000,0.013057,0.984950,0.984192,0.984899
9,0.000600,0.013084,0.984635,0.984076,0.984585
10,0.000600,0.013086,0.984635,0.984076,0.984585


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/ubuntu/.pyenv/versions/3.10.14/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not pass

TrainOutput(global_step=1150, training_loss=0.019849216297025914, metrics={'train_runtime': 18611.1639, 'train_samples_per_second': 3.068, 'train_steps_per_second': 0.062, 'total_flos': 2.941166673575117e+17, 'train_loss': 0.019849216297025914, 'epoch': 10.0})

In [21]:
# save model
from huggingface_hub import notebook_login
notebook_login()
# trainer.model.save_pretrained(peft_model_id)
# tokenizer.save_pretrained(peft_model_id)

# load model
# peft_model_id = 'multilabel_mistral'
# model = AutoModelForSequenceClassification.from_pretrained(peft_model_id)

In [22]:
peft_model_id = 'multilabel_lamma_3'
trainer.push_to_hub()

adapter_model.safetensors:   0%|          | 0.00/54.7M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shayan283/multilabel_classification/commit/0c63c1985ba7f2be97694de5556daddd8b16cb53', commit_message='End of training', commit_description='', oid='0c63c1985ba7f2be97694de5556daddd8b16cb53', pr_url=None, pr_revision=None, pr_num=None)